In [2]:
import os
import requests
from dotenv import load_dotenv

In [3]:
api_key = os.getenv("API_KEY")
city_name = 'Nairobi'
url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}"

In [ ]:
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error loading data: {response.status_code}")

{'coord': {'lon': 36.8167, 'lat': -1.2833},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 291.77,
  'feels_like': 291.83,
  'temp_min': 291.77,
  'temp_max': 291.77,
  'pressure': 1017,
  'humidity': 82,
  'sea_level': 1017,
  'grnd_level': 838},
 'visibility': 10000,
 'wind': {'speed': 3.09, 'deg': 180},
 'clouds': {'all': 75},
 'dt': 1746465164,
 'sys': {'type': 1,
  'id': 2543,
  'country': 'KE',
  'sunrise': 1746415646,
  'sunset': 1746459081},
 'timezone': 10800,
 'id': 184745,
 'name': 'Nairobi',
 'cod': 200}